In [1]:
import os
import pylandstats as pls
import geopandas as gpd
import pandas as pd
import time

wd = os.getcwd()

In [4]:
testRas = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\JoffreLakesPark\rasters\JoffreLakesPark_PACE-1994-disturbance.tif"
testZones = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\JoffreLakesPark\JoffreLakesPark_PACE.shp"

za = pls.ZonalAnalysis(testRas, masks = testZones, masks_index_col = 'ppa_gpe')
class_metrics_df = za.compute_class_metrics_df(metrics = ['proportion_of_landscape', 'total_area'], fillna = False)
class_metrics_df

,metric,proportion_of_landscape,total_area
class_val,ppa_gpe,,
0,PPA,100.0,1488.69


In [9]:
propVars = ["HMM", "disturbance", "fragstats"]

propVars = tuple([s + ".tif" for s in propVars])

disturbance.tif


In [19]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
parkFolders = parkFolders[0:1]
hmmDfs = []
files = []

for park in parkFolders:
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    ppa_gpe = os.path.join(park, parkName + "_PACE.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    for file in os.listdir(rasterLoc):
        if file.endswith(propVars):
            files.append(file)


In [4]:
dist = os.path.join(rasterLoc, files[0])
frag = os.path.join(rasterLoc, files[1])
hmm = os.path.join(rasterLoc, files[2])
print(dist, "\n", frag, "\n", hmm)

C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-2015-disturbance.tif 
 C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-2015-fragstats.tif 
 C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-2015-HMM.tif


# function for working with VLCE and disturbance data to get proportions of total area

In [11]:
def propGen(fName, subzone):
    metadata = fName.split("\\")[-1].split(".")[0].split("-")
    var = metadata[-1]
    year = metadata[1]

    za = pls.ZonalAnalysis(fName, masks = subzone, masks_index_col = 'ppa_gpe')
    class_metrics_df = za.compute_class_metrics_df(metrics = ['proportion_of_landscape', 'total_area'], fillna = False)
    class_metrics_df["park"] = parkName
    class_metrics_df["var"] = var
    class_metrics_df["year"] = year
    return class_metrics_df

# creating csv files

In [48]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
#parkFolders = parkFolders
hmmDfs = []
distDfs = []
i = 1


for park in parkFolders:
    j = 1
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    ppa_gpe = os.path.join(park, parkName + "_PACE.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    tifs = []
    for file in os.listdir(rasterLoc):
        if file.endswith(propVars):
            tifs.append(file)
    #print(tifs)
    for file in tifs:
        print(str(i), "/", len(parkFolders), "folders", str(j), "/", len(tifs), "files:", file, end = "\r")
        
        if file.endswith(propVars[0]): #vlce data
            hmmDf = propGen(os.path.join(rasterLoc, file), ppa_gpe)
            hmmDfs.append(hmmDf)
        if file.endswith(propVars[1]): #disturbance data
            
            distDf = propGen(os.path.join(rasterLoc, file), ppa_gpe)
            distDfs.append(distDf)
            
        #if file.endswith(propVars[2]):
            #files.append(file)
            #implement things for forest fragstats metrics
        
        j += 1
        time.sleep(.2)
    i += 1    

pd.concat(hmmDfs).fillna(0).to_csv("outputCsvs/vlce.csv")
pd.concat(distDfs).fillna(0).to_csv("outputCsvs/disturbance.csv")
print("done")

# function for working with reclassified VLCE data to get fragstats

In [12]:
subzone
frag

za = pls.ZonalAnalysis(frag, masks = subzone, masks_index_col = 'ppa_gpe')